In [1]:
import scipy.io as sio
import numpy as np
import theano.tensor as T
import functools

participants = []
for num in xrange(1,10):
    participants.append(sio.loadmat("data-science-P" + str(num) + ".mat"))

def convert(x, data):
    if x == 0:
        return float('0')
    else:
        return data[x-1]

processedData = {"p"+str(i): [] for i in xrange(1,10)}

for i, p in enumerate(participants):
    mapping = p["meta"]["coordToCol"][0][0]
    print 1
    for w in xrange(360):
        oriData = p["data"][w][0][0]
        vConvert = np.vectorize(lambda x: convert(x, oriData))
        processedData["p"+str(i+1)].append(vConvert(np.copy(mapping)))

for i in xrange(9):
    processedData["p"+str(i+1)] = np.array(processedData["p"+str(i+1)])

1
1
1
1
1
1
1
1
1


In [2]:
processedData["p1"].shape

(360, 51, 61, 23)

## Parsing the y of the data

In [3]:
yCond = np.empty([360, 1], dtype="int32")
yWord = np.empty([360, 1], dtype="int32")

for i in xrange(360):
    condN = participants[0]["info"]["cond_number"][0][i][0]
    wordN = participants[0]["info"]["word_number"][0][i][0]
    yCond[i] = condN
    yWord[i] = 10*condN + wordN

uniqueWord = np.unique(yWord)

for i in xrange(360):
    yWord[i] = np.where(uniqueWord==yWord[i])[0][0]

In [4]:
yCond = yCond - 1

In [5]:
yWord.dtype

dtype('int32')

In [54]:
sio.savemat('condOrWords.mat', {"cond":yCond, "word":yWord})

In [6]:
for i in xrange(9):
#     processedData["p"+str(i+1)] = np.swapaxes(processedData["p"+str(i+1)], 1, 3)
    processedData["p"+str(i+1)] = np.expand_dims(processedData["p"+str(i+1)], axis=1)

In [7]:
processedData["p1"].shape

(360, 1, 51, 61, 23)

In [8]:
from scipy.io import savemat, loadmat

savemat("PreproCNN.mat", processedData)

In [9]:
concatenatedData = reduce(lambda x,y:np.concatenate((x, y), axis=0), processedData.values()[0:9])

In [10]:
condY = np.tile(yCond,(9,1))
wordY = np.tile(yWord,(9,1))

allYs = np.concatenate((condY, wordY), axis=1)

In [11]:
allYs

array([[ 9, 42],
       [ 3, 11],
       [ 9, 40],
       ..., 
       [ 1,  2],
       [10, 48],
       [12, 56]], dtype=int32)

In [12]:
from sklearn.cross_validation import train_test_split

xTrain, xTest, yTrain, yTest = \
    train_test_split(concatenatedData, allYs, test_size=0.2, random_state=44)

In [13]:
variableList = [xTrain, xTest, yTrain, yTest]
dataReadyForCNN = {"xTrain":xTrain, "xTest":xTest, "yTrain":yTrain, "yTest":yTest}

sio.savemat("DataForCNN.mat", dataReadyForCNN)